### Deteção de anomalias em dados de gestão de resíduos em Portugal

##### João Ribeiro

## <span style="color:Darkgreen"> 1. Clustering Produtores  </span> 

In [ ]:
# Bibliotecas 
import pandas as pd
import numpy as np
import warnings
import re
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
warnings.filterwarnings("ignore")

### <span style="color:Darkgreen"> 1.1. Clustering Produtores sem CAE  </span> 

In [ ]:
df_clustering_inicial= pd.read_excel('produtores_modelos.xlsx')

In [ ]:
df_clustering_inicial.columns

In [ ]:
df_clustering_inicial.dtypes

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_clustering_inicial.head())

In [ ]:
df_clustering_inicial.info()

In [ ]:
# Substituir os valores da coluna 'ler_codigo'
df_clustering_inicial['codigos_ler_associados'] = df_clustering_inicial['codigos_ler_associados'].replace({
    'Compósitos':150105,
    'Misturas':150106 ,
    'Embalagens c/ resíduos ou contaminadas':150110
})

In [ ]:
media_quantidade_total = df_clustering_inicial['quantidade_total'].mean()
media_quantidade_total

In [ ]:
df_clustering_inicial = pd.get_dummies(df_clustering_inicial, columns=['codigos_ler_associados'])

In [ ]:
df_clustering_inicial['mercadoria_perigosa_transformada'] = np.where(df_clustering_inicial['mercadorias_perigosas'], 1, 0)

In [ ]:
df_clustering_inicial.head()

In [ ]:
desvio_padrao_quantidade_total = df_clustering_inicial['quantidade_total'].std()

# Normalização com Z-Score
df_clustering_inicial['quantidade_total_normalizada'] = df_clustering_inicial['quantidade_total'].apply(
    lambda x: (x - media_quantidade_total) / desvio_padrao_quantidade_total
)

In [ ]:
# Boxplot para a variável 'quantidade_total_normalizada' para detetar outliers
plt.figure(figsize=(8, 6))
sns.boxplot(data=df_clustering_inicial['quantidade_total_normalizada'])
plt.title('Boxplot da Quantidade total')
plt.show()

In [ ]:
df_clustering_inicial = df_clustering_inicial[df_clustering_inicial['quantidade_total_normalizada'] < 20]

In [ ]:
df_clustering_inicial.head()

In [ ]:
# Boxplot para a variável 'quantidade_total_normalizada' para detetar outliers
plt.figure(figsize=(8, 6))
sns.boxplot(data=df_clustering_inicial['quantidade_total_normalizada'])
plt.title('Boxplot da Quantidade total')
plt.show()

In [ ]:
df_clustering_inicial = df_clustering_inicial[df_clustering_inicial['quantidade_total_normalizada'] < 2]

In [ ]:
# Copia com tratamento feito
df_clustering =df_clustering_inicial.copy()

In [ ]:
# Cópia para aceder ao CAE
df_clustering_final = df_clustering[['caes_associados']].copy()

In [ ]:
# Selecionar colunas a utilizar no K-means
colunas_desejadas = ['quantidade_total_normalizada', 'mercadoria_perigosa_transformada', 'codigos_ler_associados_150105',
                     'codigos_ler_associados_150106','codigos_ler_associados_150110' ]  
df_clustering = df_clustering[colunas_desejadas]

In [ ]:
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_clustering)

In [ ]:
inertias = []

# Calcular a inércia para diferentes números de clusters
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df_scaled)
    inertias.append(kmeans.inertia_)
    
plt.figure(figsize=(8, 5))
plt.plot(range(1, 11), inertias, marker='o',color='Darkgreen')
plt.title('Gráfico do Cotovelo')
plt.xlabel('Número de Clusters')
plt.ylabel('Inércia')
plt.grid(True)
plt.show()

In [ ]:
# Escolha do número de clusters com base no gráfico anterior
k_ideal = 5
kmeans = KMeans(n_clusters=k_ideal, random_state=42)
df_clustering_final['cluster'] = kmeans.fit_predict(df_scaled)

In [ ]:
# Calcular as médias das variáveis por cluster
df_clustering['cluster'] = kmeans.labels_
media_cluster = df_clustering.groupby('cluster').mean()
media_cluster

In [ ]:
# Número de produtores alocados a cada cluster
cluster_contagem = df_clustering_final['cluster'].value_counts().sort_index()
cluster_contagem

In [ ]:
palete = sns.color_palette(["#003366",  "#0033cc","#66b3ff","#003d00","#C0C0C0",  "#66cc99",  "#800000", "#cc6600"])

### <span style="color:Darkgreen"> Mercadorias perigosas e Quantidade </span> 

In [ ]:
plt.figure(figsize=(8, 5))
sns.scatterplot(data=df_clustering, x='quantidade_total_normalizada', y='mercadoria_perigosa_transformada', hue='cluster', palette=palete, s=100, alpha=0.7)
plt.title('Clusters com Mercadorias Perigosas')
plt.xlabel('Quantidade Total')
plt.ylabel('Mercadoria Perigosa') 
plt.show()

In [ ]:
df_merc_perig = df_clustering[df_clustering['mercadoria_perigosa_transformada'] == 1]

# Contar o número de pontos em cada cluster em que é uma mercadoria Perigosa
cluster_countagem = df_merc_perig['cluster'].value_counts().reset_index()
cluster_countagem.columns = ['Cluster', 'Contagem']
cluster_countagem

### <span style="color:Darkgreen"> Compósitos (150105) </span> 

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_clustering, x='quantidade_total_normalizada', y='codigos_ler_associados_150105', hue='cluster',
                palette=palete, s=200, alpha=0.7)
plt.title('Clusters com compósitos')
plt.xlabel('Quantidade Total')
plt.ylabel('Resíduos Compósitos') 
plt.show()

In [ ]:
df_codigos_105 = df_clustering[df_clustering['codigos_ler_associados_150105'] == 1]

# Contar o número de pontos em cada cluster em que é um resíduo Compósito
cluster_contagem = df_codigos_105['cluster'].value_counts().reset_index()
cluster_contagem.columns = ['Cluster', 'Count']
cluster_contagem

In [ ]:
df_filtered = df_clustering[df_clustering['codigos_ler_associados_150105'] == 0]

 # Contar o número de pontos em cada cluster em que não é um resíduo Compósito
cluster_contagem = df_filtered['cluster'].value_counts().reset_index()
cluster_contagem.columns = ['Cluster', 'Count']
cluster_contagem

### <span style="color:Darkgreen"> Misutas (150106) </span> 

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_clustering, x='quantidade_total_normalizada', y='codigos_ler_associados_150106', hue='cluster', palette=palete, s=200, alpha=0.7)
plt.title('Clusters com Resíduos Mistura')
plt.xlabel('Quantidade Total')
plt.ylabel('Resíduos Misturas') 
plt.show()

In [ ]:
df_codigos_106 = df_clustering[df_clustering['codigos_ler_associados_150106'] == 1]
# Contar o número de pontos em cada cluster em que é uma Mistura
cluster_contagem = df_codigos_106['cluster'].value_counts().reset_index()
cluster_contagem.columns = ['Cluster', 'Count']
cluster_contagem

### <span style="color:Darkgreen"> Embalagens c/ resíduos ou contaminadas (150110) </span> 

In [ ]:
plt.figure(figsize=(12, 6))
sns.scatterplot(data=df_clustering, x='quantidade_total_normalizada', y='codigos_ler_associados_150110', hue='cluster', 
                palette=palete, s=200, alpha=0.7)
plt.title('Clusters Embalagens c/ resíduos ou contaminadas e Quantidade')
plt.xlabel('Quantidade Total')
plt.ylabel('Embalagens c/ resíduos ou contaminadas ')
plt.show()

In [ ]:
df_codigos_110 = df_clustering[df_clustering['codigos_ler_associados_150110'] == 1]
# Contar o número de pontos em cada cluster em que é uma Embalagem c/ resíduos ou contaminadas
cluster_contagem = df_codigos_110['cluster'].value_counts().reset_index()
cluster_contagem.columns = ['Cluster', 'Count']
cluster_contagem

### <span style="color:Darkgreen"> Medidas Descritivas do Cluster </span> 

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(data=df_clustering, x='cluster', color='Darkgreen')
plt.title('Número de Pontos em Cada Cluster')
plt.show()

In [ ]:
# Medidas Descritivas para cada cluster
Estatistica = df_clustering.groupby('cluster').agg({
    'quantidade_total_normalizada': ['mean', 'min', 'max'],
    'mercadoria_perigosa_transformada': ['mean'],
    'codigos_ler_associados_150105': ['mean'],
    'codigos_ler_associados_150106': ['mean'],
    'codigos_ler_associados_150110': ['mean']
}).reset_index()

print("Estatísticas descritivas por cluster:")
Estatistica

In [ ]:
# Mapa de calor
plt.figure(figsize=(14, 8))
ax = sns.heatmap(media_cluster, annot=True, cmap='Greens', cbar=True)
plt.title('Médias das Variáveis por Cluster', fontsize=14)
plt.xlabel('Variáveis', fontsize=12)
plt.ylabel('Clusters', fontsize=12)
ax.set_xticklabels(['Quantidade Total', 'Mercadoria Perigosa', 'Resíduos Compósitos', 
                    'Resíduos Mistura', 'Resíduos Contaminados'], rotation=45, ha='right', fontsize=8)
ax.set_yticklabels(ax.get_yticklabels(), ha='right', fontsize=8)
plt.show()

### <span style="color:Darkgreen"> PCA </span> 

In [ ]:
pca = PCA(n_components=2) 
df_pca = pca.fit_transform(df_scaled)

# Componentes principais e clusters
df_pca = pd.DataFrame(df_pca, columns=['PC1', 'PC2'])
df_pca['cluster'] = kmeans.predict(df_scaled)  # Adicionar a coluna de clusters

plt.figure(figsize=(12, 8))
sns.scatterplot(x='PC1', y='PC2', hue='cluster', data=df_pca, palette=palete, edgecolor='w', s=100)
plt.title('PCA dos Dados com KMeans Clustering')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend(title='Cluster')
plt.grid(True)
plt.show()

In [ ]:
silhouette_avg = silhouette_score(df_scaled, df_clustering['cluster'])
print(f'Silhouette Score médio: {silhouette_avg}')

**Nota**: valor próximo de 1 indica que as amostras estão muito bem agrupadas dentro de seus próprios clusters e bem separadas dos outros clusters.

##### <span style="color:Darkgreen"> CAES associados aos clusters da experiencia 1 - Produtores </span> 

In [ ]:
# Função para extrair o dígito mais frequente
def digito_mais_frequente(linha):
    # Filtrar e extrair o primeiro dígito de cada número, ignorando "Inexistente"
    numeros = [num for num in str(linha).split(', ') if num != 'Inexistente']
    
    if not numeros:  # Se a lista de números estiver vazia, retorna None
        return None
    
    primeiros_digitos = [str(int(float(num)))[0] for num in numeros]
    
    # Contar a frequência de cada dígito
    contagem = Counter(primeiros_digitos)
    return contagem.most_common(1)[0][0] if contagem else None


In [ ]:
# Aplicar a função na coluna 'caes_associados' para substituí-la diretamente
df_clustering_final['caes_associados'] = df_clustering_final['caes_associados'].apply(digito_mais_frequente)

# Remover as linhas onde 'caes_associados' é None (ou seja, quando só havia "Inexistente")
df_clustering_final.dropna(subset=['caes_associados'], inplace=True)
df_clustering_final[['caes_associados']]

In [ ]:
top_cae_por_cluster = df_clustering_final.groupby('cluster')['caes_associados'].apply(lambda x: x.value_counts().head(3)).reset_index()
top_cae_por_cluster.columns = ['cluster', 'cae', 'contagem']
top_cae_por_cluster


### <span style="color:Darkgreen"> 1.2. Clustering de Produtores com CAE </span> 

In [ ]:
df_cae=df_clustering_inicial.copy()

In [ ]:
df_cae.head()

In [ ]:
# Aplicar a função na coluna 'caes_associados' para substituí-la diretamente
df_cae['caes_associados'] = df_cae['caes_associados'].apply(digito_mais_frequente)

# Remover as linhas onde 'caes_associados' é None (ou seja, quando só havia "Inexistente")
df_cae.dropna(subset=['caes_associados'], inplace=True)
df_cae[['caes_associados']]


In [ ]:
df_cae.head()

In [ ]:
df_cae = pd.get_dummies(df_cae, columns=['caes_associados'])

In [ ]:
df_cae.head()

In [ ]:
# Selecionar colunas a utilizar no K-means
colunas_desejadas = [ 'quantidade_total_normalizada', 'mercadoria_perigosa_transformada','codigos_ler_associados_150105',
                     'codigos_ler_associados_150106','codigos_ler_associados_150110','caes_associados_1','caes_associados_2',
                     'caes_associados_3','caes_associados_4','caes_associados_5',
                     'caes_associados_6','caes_associados_7','caes_associados_8','caes_associados_9' ]  
df_novo = df_cae[colunas_desejadas]

In [ ]:
scaler = StandardScaler()
df_scaled_novo = scaler.fit_transform(df_novo)

In [ ]:
inertias = []
for k in range(5, 21):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df_scaled_novo)
    inertias.append(kmeans.inertia_)
    
plt.figure(figsize=(8, 5))
plt.plot(range(5, 21), inertias, marker='o',color='darkgreen')
plt.title('Gráfico do Cotovelo')
plt.xlabel('Número de Clusters')
plt.ylabel('Inércia')
plt.grid(True)
plt.show()

In [ ]:
k_ideal = 12
kmeans = KMeans(n_clusters=k_ideal, random_state=42)
df_novo['cluster'] = kmeans.fit_predict(df_scaled_novo)

In [ ]:
silhouette_avg_novo = silhouette_score(df_scaled_novo, df_novo['cluster'])
print(f'Silhouette Score médio: {silhouette_avg_novo}')

In [ ]:
# Calcular as médias das variáveis por cluster
cluster_means2 = df_novo.groupby('cluster').mean()
cluster_means2

In [ ]:
cluster_contagem2 = df_novo['cluster'].value_counts().sort_index()
cluster_contagem2

**Signficado do primeiro digito do CAE (Principais Industrias):**	

    1. Agricultura
	2. Indústrias Extrativas
	3. Transformação
	4. Energia, Agua, Resíduos
	5. Construção
	6. Automoveis
	7. Transportes
	8. Alimentação
    9. Comunicação

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(data=df_novo, x='cluster', color='Darkgreen')
plt.title('Número de Pontos em Cada Cluster')
plt.xlabel('Número do Cluster')
plt.ylabel('Contagem de Produtores')
plt.show()

In [ ]:
plt.figure(figsize=(14, 8))

# Gera o heatmap
ax = sns.heatmap(cluster_means2, annot=True, cmap='Greens', cbar=True)

plt.title('Médias das Variáveis por Cluster')

# Obtém o número de colunas do teu dataset
num_cols = cluster_means2.shape[1]
ax.set_xticks([x + 0.5 for x in range(num_cols)])  
ax.set_xticklabels(['Quantidade Total', 'Mercadoria Perigosa', 'Resíduos Compósitos', 
                    'Resíduos Mistura', 'Emb c/ resíduos ou Contaminadas', 'Agricultura, Animais e Pesca', 
                    'Industrias Extrativas', 'Indústrias Transformadoras', 'Eletricidade, Gás, Vapor, Água ', 
                    'Águas e Resíduos', 'Construção', 'Comércio e Automóveis', 
                    'Transportes e Armazenamento', 'Alojamento, Restauração e Similares'], 
                   rotation=45, ha='right', fontsize=8)
plt.show()


## <span style="color:Darkgreen"> 2. Clustering Transformadores  </span> 

In [ ]:
# Bibliotecas 
import pandas as pd
import numpy as np
import warnings
import re
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
warnings.filterwarnings("ignore")

### <span style="color:Darkgreen"> 2.1. Clustering Transformadores sem CAE  </span> 

In [ ]:
df_clustering2_inicial = pd.read_excel('transformadores_modelos.xlsx')

In [ ]:
df_clustering2_inicial.columns

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_clustering2_inicial.head())

In [ ]:
# Substituir os valores da coluna 'ler_codigo'
df_clustering2_inicial['codigos_ler_associados'] = df_clustering2_inicial['codigos_ler_associados'].replace({
    'Compósitos':150105,
    'Misturas':150106 ,
    'Embalagens c/ resíduos ou contaminadas':150110
})

In [ ]:
media_quantidade_total2 = df_clustering2_inicial['quantidade_total'].mean()
media_quantidade_total2

In [ ]:
df_clustering2_inicial = pd.get_dummies(df_clustering2_inicial, columns=['codigos_ler_associados'])

In [ ]:
df_clustering2_inicial['mercadoria_perigosa_transformada'] = np.where(df_clustering2_inicial['mercadorias_perigosas'], 1, 0)

In [ ]:
df_clustering2_inicial.head()

In [ ]:
desvio_padrao_quantidade_total2 = df_clustering2_inicial['quantidade_total'].std()

# Normalização com Z-Score
df_clustering2_inicial['quantidade_total_normalizada'] = df_clustering2_inicial['quantidade_total'].apply(
    lambda x: (x - media_quantidade_total2) / desvio_padrao_quantidade_total2
)

In [ ]:
df_clustering2_inicial.head()

In [ ]:
# Boxplot para a variável 'quantidade_total_normalizada' para detetar outliers
plt.figure(figsize=(8, 6))
sns.boxplot(data=df_clustering2_inicial['quantidade_total_normalizada'])
plt.title('Boxplot da Quantidade total Recebida')
plt.show()

In [ ]:
df_clustering2_inicial = df_clustering2_inicial[df_clustering2_inicial['quantidade_total_normalizada'] < 5]

In [ ]:
# Boxplot para a variável 'quantidade_total_normalizada' para detetar outliers
plt.figure(figsize=(8, 6))
sns.boxplot(data=df_clustering2_inicial['quantidade_total_normalizada'])
plt.title('Boxplot da Quantidade total Recebida')
plt.show()

In [ ]:
df_clustering2_inicial = df_clustering2_inicial[df_clustering2_inicial['quantidade_total_normalizada'] < 2]

In [ ]:
df_clustering2=df_clustering2_inicial.copy()

In [ ]:
df_clustering_final2 = df_clustering2[['caes_associados']].copy()

In [ ]:
# Selecionar colunas a utilizar no K-means
colunas_desejadas = ['quantidade_total_normalizada', 'mercadoria_perigosa_transformada', 'codigos_ler_associados_150105',
                     'codigos_ler_associados_150106','codigos_ler_associados_150110' ]  
df_clustering2 = df_clustering2[colunas_desejadas]

In [ ]:
scaler = StandardScaler()
df_scaled2 = scaler.fit_transform(df_clustering2)

In [ ]:
inertias = []

# Calcular a inércia para diferentes números de clusters
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df_scaled2)
    inertias.append(kmeans.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(range(1, 11), inertias, marker='o',color='Darkgreen')
plt.title('Gráfico do Cotovelo')
plt.xlabel('Número de Clusters')
plt.ylabel('Inércia')
plt.grid(True)
plt.show()

In [ ]:
# Escolha do número de clusters com base no gráfico anterior
k_ideal = 6
kmeans = KMeans(n_clusters=k_ideal, random_state=42)
df_clustering_final2['cluster'] = kmeans.fit_predict(df_scaled2)

In [ ]:
# Calcular as médias das variáveis por cluster
df_clustering2['cluster'] = kmeans.labels_
media_cluster2 = df_clustering2.groupby('cluster').mean()
media_cluster2

In [ ]:
# Número de transformadores alocados a cada cluster
cluster_contagem2 = df_clustering_final2['cluster'].value_counts().sort_index()
cluster_contagem2

In [ ]:
palete = sns.color_palette(["#003366",  "#0033cc","#66b3ff","#003d00","#C0C0C0",  "#66cc99",  "#800000", "#cc6600"])

### <span style="color:Darkgreen"> Mercadorias perigosas e Quantidade Recebida </span> 

In [ ]:
plt.figure(figsize=(8, 5))
sns.scatterplot(data=df_clustering2, x='quantidade_total_normalizada', y='mercadoria_perigosa_transformada', hue='cluster', palette=palete, s=100, alpha=0.7)
plt.title('Clusters com Mercadorias Perigosas')
plt.xlabel('Quantidade Total')
plt.ylabel('Mercadoria Perigosa') 
plt.show()

In [ ]:
df_merc_perig2 = df_clustering2[df_clustering2['mercadoria_perigosa_transformada'] == 1]

# Contar o número de pontos em cada cluster em que é uma mercadoria Perigosa
cluster_countagem2 = df_merc_perig2['cluster'].value_counts().reset_index()
cluster_countagem2.columns = ['Cluster', 'Contagem']
cluster_countagem2

### <span style="color:Darkgreen"> Compósitos (150105) </span> 

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_clustering2, x='quantidade_total_normalizada', y='codigos_ler_associados_150105', hue='cluster',
                palette=palete, s=200, alpha=0.7)
plt.title('Clusters com compósitos')
plt.xlabel('Quantidade Total')
plt.ylabel('Resíduos Compósitos') 
plt.show()

In [ ]:
df_codigos2_105 = df_clustering2[df_clustering2['codigos_ler_associados_150105'] == 1]
# Contar o número de pontos em cada cluster em que é um resíduo Compósito
cluster_contagem2 = df_codigos2_105['cluster'].value_counts().reset_index()
cluster_contagem2.columns = ['Cluster', 'Count']
cluster_contagem2

In [ ]:
df_filtered2 = df_clustering2[df_clustering2['codigos_ler_associados_150105'] == 0]

 # Contar o número de pontos em cada cluster em que não é um resíduo Compósito
cluster_contagem2 = df_filtered2['cluster'].value_counts().reset_index()
cluster_contagem2.columns = ['Cluster', 'Count']
cluster_contagem2

### <span style="color:Darkgreen"> Misutas (150106) </span> 

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_clustering2, x='quantidade_total_normalizada', y='codigos_ler_associados_150106', hue='cluster', palette=palete, s=200, alpha=0.7)
plt.title('Clusters com Resíduos Mistura')
plt.xlabel('Quantidade Total')
plt.ylabel('Resíduos Misturas') 
plt.show()

In [ ]:
df_codigos2_106 = df_clustering2[df_clustering2['codigos_ler_associados_150106'] == 1]
 # Contar o número de pontos em cada cluster em que é uma Mistura
cluster_contagem2 = df_codigos2_106['cluster'].value_counts().reset_index()
cluster_contagem2.columns = ['Cluster', 'Count']
cluster_contagem2

### <span style="color:Darkgreen"> Embalagens c/ resíduos ou contaminadas (150110) </span> 

In [ ]:
plt.figure(figsize=(12, 6))
sns.scatterplot(data=df_clustering2, x='quantidade_total_normalizada', y='codigos_ler_associados_150110', hue='cluster', 
                palette=palete, s=200, alpha=0.7)
plt.title('Clusters Embalagens c/ resíduos ou contaminadas e Quantidade')
plt.xlabel('Quantidade Total')
plt.ylabel('Embalagens c/ resíduos ou contaminadas ')
plt.show()

In [ ]:
df_codigos2_110 = df_clustering2[df_clustering2['codigos_ler_associados_150110'] == 1]
# Contar o número de pontos em cada cluster em que é uma Embalagem c/ resíduos ou contaminadas
cluster_contagem2 = df_codigos2_110['cluster'].value_counts().reset_index()
cluster_contagem2.columns = ['Cluster', 'Count']
cluster_contagem2

### <span style="color:Darkgreen"> Medidas Descritivas do Cluster </span> 

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(data=df_clustering2, x='cluster', color='Darkgreen')
plt.title('Número de Pontos em Cada Cluster')
plt.show()

In [ ]:
# Medidas Descritivas para cada cluster
Estatistica2 = df_clustering2.groupby('cluster').agg({
    'quantidade_total_normalizada': ['mean', 'min', 'max'],
    'mercadoria_perigosa_transformada': ['mean'],
    'codigos_ler_associados_150105': ['mean'],
    'codigos_ler_associados_150106': ['mean'],
    'codigos_ler_associados_150110': ['mean']
}).reset_index()

print("Estatísticas descritivas por cluster:")
Estatistica2

In [ ]:
# Mapa de calor
plt.figure(figsize=(14, 8))
ax = sns.heatmap(media_cluster2, annot=True, cmap='Greens', cbar=True)
plt.title('Médias das Variáveis por Cluster', fontsize=14)
plt.xlabel('Variáveis', fontsize=12)
plt.ylabel('Clusters', fontsize=12)
ax.set_xticklabels(['Quantidade Total', 'Mercadoria Perigosa', 'Resíduos Compósitos', 
                    'Resíduos Mistura', 'Resíduos Contaminados'], rotation=45, ha='right', fontsize=8)
ax.set_yticklabels(ax.get_yticklabels(), ha='right', fontsize=8)
plt.show()

### <span style="color:Darkgreen"> PCA </span> 

In [ ]:
pca = PCA(n_components=2) 
df_pca2 = pca.fit_transform(df_scaled2)

# Componentes principais e clusters
df_pca2 = pd.DataFrame(df_pca2, columns=['PC1', 'PC2'])
df_pca2['cluster'] = kmeans.predict(df_scaled2)  # Adicionar a coluna de clusters

plt.figure(figsize=(12, 8))
sns.scatterplot(x='PC1', y='PC2', hue='cluster', data=df_pca2, palette=palete, edgecolor='w', s=100)
plt.title('PCA dos Dados com KMeans Clustering')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend(title='Cluster')
plt.grid(True)
plt.show()

In [ ]:
silhouette_avg2 = silhouette_score(df_scaled2, df_clustering2['cluster'])
print(f'Silhouette Score médio: {silhouette_avg2}')

In [ ]:
# Aplicar a função na coluna 'caes_associados' para substituí-la diretamente
df_clustering_final2['caes_associados'] = df_clustering_final2['caes_associados'].apply(digito_mais_frequente)

# Remover as linhas onde 'caes_associados' é None (ou seja, quando só havia "Inexistente")
df_clustering_final2.dropna(subset=['caes_associados'], inplace=True)
df_clustering_final2[['caes_associados']]

In [ ]:
top_cae_por_cluster2 = df_clustering_final2.groupby('cluster')['caes_associados'].apply(lambda x: x.value_counts().head(3)).reset_index()
top_cae_por_cluster2.columns = ['cluster', 'cae', 'contagem']
top_cae_por_cluster2

### <span style="color:Darkgreen">2.2. Clustering de Transformadores com CAE </span> 

In [ ]:
df_cae2=df_clustering2_inicial.copy()

In [ ]:
df_cae2.head()

In [ ]:
# Aplicar a função na coluna 'caes_associados' para substituí-la diretamente
df_cae2['caes_associados'] = df_cae2['caes_associados'].apply(digito_mais_frequente)
df_cae2.dropna(subset=['caes_associados'], inplace=True)
df_cae2[['caes_associados']]


In [ ]:
df_cae2.head()

In [ ]:
df_cae2.info()

In [ ]:
df_cae2 = pd.get_dummies(df_cae2, columns=['caes_associados'])

In [ ]:
# Selecionar colunas a utilizar no K-means
colunas_desejadas2 = ['quantidade_total_normalizada', 'mercadoria_perigosa_transformada', 'codigos_ler_associados_150105',
                     'codigos_ler_associados_150106','codigos_ler_associados_150110','caes_associados_1','caes_associados_2',
                     'caes_associados_3','caes_associados_4','caes_associados_5',
                     'caes_associados_6','caes_associados_7','caes_associados_8','caes_associados_9' ]  
df_transf = df_cae2[colunas_desejadas2]

In [ ]:
scaler2 = StandardScaler()
df_scaled_novo2 = scaler.fit_transform(df_transf)

In [ ]:
inertias = []
for k in range(1, 21):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df_scaled_novo2)
    inertias.append(kmeans.inertia_)
    
plt.figure(figsize=(8, 5))
plt.plot(range(1, 21), inertias, marker='o',color='darkgreen')
plt.title('Gráfico do Cotovelo')
plt.xlabel('Número de Clusters')
plt.ylabel('Inércia')
plt.grid(True)
plt.show()

In [ ]:
k_ideal2 = 12
kmeans2 = KMeans(n_clusters=k_ideal2, random_state=42)
df_transf['cluster'] = kmeans2.fit_predict(df_scaled_novo2)

In [ ]:
silhouette_avg_novo2 = silhouette_score(df_scaled_novo2, df_transf['cluster'])
print(f'Silhouette Score médio: {silhouette_avg_novo2}')

In [ ]:
# Calcular as médias das variáveis por cluster
cluster_means2_2 = df_transf.groupby('cluster').mean()
cluster_means2_2

In [ ]:
cluster_contagem2_2 = df_transf['cluster'].value_counts().sort_index()
cluster_contagem2_2

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(data=df_transf,x='cluster', color='Darkgreen')
plt.title('Número de Pontos em Cada Cluster')
plt.xlabel('Número do Cluster')
plt.ylabel('Contagem de Produtores')
plt.show()

In [ ]:
plt.figure(figsize=(14, 8))
ax= sns.heatmap(cluster_means2_2, annot=True, cmap='Greens', cbar=True)
plt.title('Médias das Variáveis por Cluster')
num_cols = cluster_means2_2.shape[1]
ax.set_xticks([x + 0.5 for x in range(num_cols)])  
ax.set_xticklabels(['Quantidade Total', 'Mercadoria Perigosa', 'Resíduos Compósitos', 
                    'Resíduos Mistura', 'Resíduos Contaminados', 'Agricultura, Animais e Pesca', 
                    'Industrias Extrativas', 'Indústrias Transformadoras', 'Eletricidade, Gás, Vapor, Água ', 
                    'Águas e Resíduos', 'Construção', 'Comércio e Automóveis', 
                    'Transportes e Armazenamento', 'Alojamento, Restauração e Similares'], 
                   rotation=45, ha='right', fontsize=8)
plt.show()